In [ ]:
from google.colab import auth
from datetime import datetime
auth.authenticate_user()
!gsutil -m cp -r gs://predict_cfs/distance3/gage_data . 

Copying gs://predict_cfs/distance3/gage_data/10011500stations.json...
Copying gs://predict_cfs/distance3/gage_data/10015700stations.json...
Copying gs://predict_cfs/distance3/gage_data/10016900stations.json...
Copying gs://predict_cfs/distance3/gage_data/10015900stations.json...
Copying gs://predict_cfs/distance3/gage_data/10020100stations.json...
Copying gs://predict_cfs/distance3/gage_data/10023000stations.json...
Copying gs://predict_cfs/distance3/gage_data/10020300stations.json...
Copying gs://predict_cfs/distance3/gage_data/10028500stations.json...
Copying gs://predict_cfs/distance3/gage_data/10026500stations.json...
Copying gs://predict_cfs/distance3/gage_data/10032000stations.json...
Copying gs://predict_cfs/distance3/gage_data/10038000stations.json...
Copying gs://predict_cfs/distance3/gage_data/10041000stations.json...
Copying gs://predict_cfs/distance3/gage_data/10104700stations.json...
Copying gs://predict_cfs/distance3/gage_data/10105900stations.json...
Copying gs://predict

In [ ]:
from math import radians, cos, sin, asin, sqrt
import pandas as pd
import os
import json
from typing import Set, Tuple, Dict
import requests
from datetime import datetime, timedelta

def get_closest_gage(gage_df:pd.DataFrame, station_df:pd.DataFrame, path_dir:str, start_row:int, end_row:int):
  # Function that calculates the closest weather stations to gage and stores in JSON
  # Base u
  for row in range(start_row, end_row):
    gage_info = {}
    gage_info["river_id"] = int(gage_df.iloc[row]['id'])
    gage_lat = gage_df.iloc[row]['latitude']
    gage_long = gage_df.iloc[row]['logitude']
    gage_info["stations"] = []
    total = len(station_df.index)
    for i in range(0, total):
      stat_row = station_df.iloc[i]
      dist = haversine(stat_row["lon"], stat_row["lat"], gage_long, gage_lat)
      st_id = stat_row['stid']
      gage_info["stations"].append({"station_id":st_id, "dist":dist})
    gage_info["stations"] = sorted(gage_info['stations'], key = lambda i: i["dist"], reverse=True) 
    with open(os.path.join(path_dir, str(gage_info["river_id"]) + "stations.json"), 'w') as w:
      json.dump(gage_info, w)
      if count%100 == 0:
        print("Currently at " + str(count))
      count +=1 

def haversine(lon1, lat1, lon2, lat2):
  """
  Calculate the great circle distance between two points 
  on the earth (specified in decimal degrees)
  """
  # convert decimal degrees to radians 
  lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

  # haversine formula 
  dlon = lon2 - lon1 
  dlat = lat2 - lat1 
  a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
  c = 2 * asin(sqrt(a)) 
  r = 6371 # Radius of earth in kilometers. Use 3956 for miles
  return c * r

def create_visited():
  visited_gages = {"stations_visited":{}, "saved_complete":{}}
  with open("visited_gages.json", "w+") as f:
    json.dump(visited_gages,f)

def get_weather_data(file_path:str, econet_gages:Set, base_url:str, visited_path:str):
  """
  Function that retrieves if station has weather 
  data for a specific gage either from ASOS or ECONet 
  """
  # Base URL "https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station={}&data=tmpf&data=p01m&year1=2019&month1=1&day1=1&year2=2019&month2=1&day2=2&tz=Etc%2FUTC&format=onlycomma&latlon=no&missing=M&trace=T&direct=no&report_type=1&report_type=2"
  with open(visited_path, "r") as g:
    stations_explored = json.load(g)
  
  gage_meta_info = {}
  try:
    with open(file_path) as f:
      gage_data = json.load(f)
  except:
    print(file_path)
    return "Invalid"
  
  gage_meta_info["gage_id"] = gage_data["river_id"]
  gage_meta_info["stations"] = []
  closest_stations = gage_data["stations"][-20:]
  for station in reversed(closest_stations):
    if station["station_id"] in stations_explored["stations_visited"]:
      gage_meta_info["stations"].append({"station_id":station["station_id"], 
                                      "dist":station["dist"], "cat":"ASOS"})
      stations_explored["stations_visited"][str(station["station_id"])] = "True"
    else:
      url = base_url.format(station["station_id"])
      response = requests.get(url)
      if len(response.text)>100:
        gage_meta_info["stations"].append({"station_id":station["station_id"], 
                                          "dist":station["dist"], "cat":"ASOS"})
      elif station["station_id"] in econet_gages:
        gage_meta_info["stations"].append({"station_id":station["station_id"], 
                                          "dist":station["dist"], "cat":"ECO"})
  with open(visited_path, "w+") as c:
    json.dump(stations_explored, c)
  with open(file_path, "w+") as f:
    json.dump(gage_meta_info, f)

  return gage_meta_info

def format_dt(date_time_str:str) -> datetime:
  proper_datetime = datetime.strptime(date_time_str, "%Y-%m-%d %H:%M")
  if proper_datetime.minute != 0:
    proper_datetime = proper_datetime + timedelta(hours=1)
    proper_datetime = proper_datetime.replace(minute=0)
  return proper_datetime
    
def process_asos_data(file_path:str, base_url:str, stations_explored_path, client)->Dict:
  """
  Function that saves the ASOS data to CSV 
  uses output of get weather data.
  """
  station_meta_dict = {}
  with open (stations_explored_path) as g:
    stations_explored = json.load(g)
  with open(file_path) as f:
    gage_data = json.load(f)
    stations_list = []
    for station in gage_data["stations"]:
      if station["station_id"] in stations_explored["saved_complete"]:
        station1 = stations_explored["saved_complete"][station["station_id"]]
        station["missing_temp"] = station1["missing_temp"]
        station["missing_precip"] = station1["missing_precip"]
      elif station["cat"] == "ASOS" :
        response = requests.get(base_url.format(station["station_id"]))
        with open("temp_weather_data.csv", "w+") as f:
          f.write(response.text)
        df, missing_precip, missing_temp = process_asos_csv("temp_weather_data.csv")
        station["missing_precip"] = missing_precip
        station["missing_temp"] = missing_temp
        stations_explored["saved_complete"][str(station["station_id"])] = station
        df.to_csv(str(station["station_id"])+".csv")
        name = str(station["station_id"])+".csv"
        upload_file("predict_cfs",  "asos/" + name, name, client)
        station_meta_dict[station["station_id"]] = station
      stations_list.append(station)
    gage_data["stations"] = stations_list
    print(gage_data)
  with open(file_path, "w") as f:
    json.dump(gage_data, f)
  upload_file("predict_cfs", "meta_data/" + file_path, file_path, client)
  with open (stations_explored_path, "w+") as g:
    json.dump(stations_explored, g)
  upload_file("predict_cfs", "asos/" +"record.json", stations_explored_path, client)
  return gage_data

def process_asos_csv(path:str):
    df = pd.read_csv(path)
    missing_precip = df['p01m'][df['p01m']=='M'].count()
    missing_temp = df['tmpf'][df['tmpf']=='M'].count()
    df['hour_updated'] = df['valid'].map(format_dt)
    df['tmpf'] = pd.to_numeric(df['tmpf'], errors='coerce')
    
    df['p01m'] = pd.to_numeric(df['p01m'], errors='coerce')
    # Replace mising values with an average of the two closest values
    # Since stations record at different intervals this could 
    # actually cause an overestimation of precip. Instead replace with 0
    #df['p01m']=(df['p01m'].fillna(method='ffill') + df['p01m'].fillna(method='bfill'))/2
    df['p01m'] = df['p01m'].fillna(0)
    df['tmpf']=(df['tmpf'].fillna(method='ffill') + df['tmpf'].fillna(method='bfill'))/2
    df = df.groupby(by=['hour_updated'], as_index=False).agg({'p01m': 'sum', 'valid': 'first', 'tmpf': 'mean'})
    return df, int(missing_precip), int(missing_temp)


In [ ]:
eco_gage_set = {'A', 'AURO', 'BEAR', 'BUCK', 'BURN', 'CAST', 'CLA2',
 'CLAY', 'CLIN', 'DURH', 'FLET', 'FRYI', 'GARM', 'GATH', 'GBAX', 'GBRA',
 'GBRE','GBYR','GCAM', 'GCHA', 'GCHS','GCML', 'GCOH', 'GCOO', 'GDAL',
 'GDAW','GDIN', 'GFTB','GLOU','GMCI','GMCR','GMET','GMID','GNEW',
 'GOCO', 'GOKE', 'GOLD', 'GPLA','GSTA','GSTE','GTAL','GTOC','GWAS',
 'GWAY', 'HAML', 'JACK', 'JEFF', 'K0A9', 'K1A5', 'K3J7', 'K6A1',
 'K6A2', 'KACJ',
 'KAFP',
 'KAIK',
 'KARW',
 'KASJ',
 'KAYS',
 'KBBP',
 'KBGE',
 'KBGF',
 'KBIJ',
 'KBNL',
 'KBQK',
 'KCCO',
 'KCDN',
 'KCKF',
 'KCKI',
 'KCNI',
 'KCPC',
 'KCQW',
 'KCTJ',
 'KCTZ',
 'KCWV',
 'KDBN',
 'KDCM',
 'KDNN',
 'KDPL',
 'KDQH',
 'KDYB',
 'KDYR',
 'KEDE',
 'KEHO',
 'KEOE',
 'KETC',
 'KEXX',
 'KEYF',
 'KEZM',
 'KFBG',
 'KFDW',
 'KFFA',
 'KFQD',
 'KGEV',
 'KGGE',
 'KGSB',
 'KGWW',
 'KGYH',
 'KHBI',
 'KHFF',
 'KHNZ',
 'KHOE',
 'KHQU',
 'KHRJ',
 'KHVS',
 'KHXD',
 'KHYW',
 'KIIY',
 'KINS',
 'KIPJ',
 'KISO',
 'KIXA',
 'KJES',
 'KJNX',
 'KJQF',
 'KJYL',
 'KJZI',
 'KLGC',
 'KLHW',
 'KLHZ',
 'KLKR',
 'KLQK',
 'KLRO',
 'KLSF',
 'KLUX',
 'KLZU',
 'KMAO',
 'KMGE',
 'KMGR',
 'KMKS',
 'KMLJ',
 'KMMI',
 'KMMT',
 'KMNI',
 'KMQI',
 'KMQY',
 'KMRN',
 'KMWK',
 'KMYR',
 'KNBC',
 'KNBT',
 'KNCA',
 'KNJM',
 'KNKT',
 'KNQA',
 'KOAJ',
 'KOCW',
 'KONX',
 'KOPN',
 'KPGV',
 'KPOB',
 'KRBW',
 'KRCZ',
 'KRHP',
 'KRUQ',
 'KRYY',
 'KSBO',
 'KSCR',
 'KSIF',
 'KSMS',
 'KSOP',
 'KSPA',
 'KSSC',
 'KSUT',
 'KSVH',
 'KSVN',
 'KTBR',
 'KTDF',
 'KTHA',
 'KTMA',
 'KTNB',
 'KTTA',
 'KTVI',
 'KUCY',
 'KUDG',
 'KUKF',
 'KVAD',
 'KVDI',
 'KVUJ',
 'KWDR',
 'KWRB',
 'LAKE',
 'LAUR',
 'LEWS',
 'LILE',
 'MITC',
 'N7MR',
 'NBAC',
 'NBFT',
 'NBUS',
 'NCAS',
 'NCAT',
 'NCDI',
 'NCHE',
 'NCOW',
 'NCRN',
 'NDAV',
 'NDBR',
 'NDUK',
 'NELI',
 'NEWL',
 'NFAI',
 'NFBR',
 'NFIN',
 'NGRC',
 'NGRF',
 'NGUI',
 'NHIG',
 'NHOF',
 'NJCY',
 'NJES',
 'NLEX',
 'NMTI',
 'NNAC',
 'NNCP',
 'NNWB',
 'NPOC',
 'NRAV',
 'NRCK',
 'NREN',
 'NRUT',
 'NSND',
 'NSUN',
 'NTUR',
 'NTUS',
 'NTYL',
 'NUWH',
 'NWAY',
 'NWHI',
 'OXFO',
 'PLYM',
 'REED',
 'REID',
 'ROCK',
 'SALI',
 'SANP',
 'SCAR',
 'SCON',
 'SILR',
 'SKIN',
 'SPRU',
 'SSAT',
 'SSAV',
 'SSV1',
 'SSV2',
 'SWHI',
 'SWIT',
 'Station_Metadata.txt',
 'TAYL',
 'TBIG',
 'TBLE',
 'TBUR',
 'TCAM',
 'TCHC',
 'TCHE',
 'TCHU',
 'TCOK',
 'TCRO',
 'TFCL',
 'THAM',
 'TIND',
 'TLEW',
 'TMEI',
 'TMER',
 'TPRE',
 'TSHI',
 'TTOW',
 'TWES',
 'WAYN',
 'WHIT',
 'WILD',
 'WILL',
 'WINE'}

In [ ]:
from google.cloud import storage
def get_client():
  storage_client = storage.Client(project="gmap-997")
  print(storage_client.__dict__)
  # Make an authenticated API request
  buckets = list(storage_client.list_buckets())
  return storage_client

def build_dataset(json_full_path, asos_base_url, base_url_2, econet_data, visited_gages_path, start=0, end_index=100):
  directory = os.fsencode(json_full_path)
  sorted_list = sorted(os.listdir(directory))
  client = get_client()
  for i in range(start, end_index):
    file = sorted_list[i]
    filename = os.fsdecode(file)
    if get_weather_data(os.path.join(json_full_path, filename), econet_data, asos_base_url, visited_gages_path) != "Invalid":
      process_asos_data(os.path.join(json_full_path, filename), base_url_2, visited_gages_path, client)


    

In [ ]:
def upload_file(bucket_name:str, file_name:str, upload_name:str, client):
    print(upload_name)
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(file_name)
    blob.upload_from_filename(upload_name)

In [ ]:
!gsutil ls gs://predict_cfs/meta_data/distance3/** | wc -l
!gsutil cp gs://predict_cfs/asos/record.json .
!gsutil ls gs://predict_cfs/meta2 | wc -l
!gsutil ls gs://predict_cfs/meta_data/gage_data | wc -l

#1138, 1139

3880
Copying gs://predict_cfs/asos/record.json...
/ [1 files][376.8 KiB/376.8 KiB]                                                
Operation completed over 1 objects/376.8 KiB.                                    
1269
5171


In [ ]:
!gsutil ls gs://predict_cfs/meta3 | wc -l
!gsutil ls gs://predict_cfs/joined2 | wc -l
!gsutil ls gs://predict_cfs/joined | wc -l
!gsutil ls gs://predict_cfs/joined5 | wc -l

247
246
1266
596


In [ ]:
base_url = "https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station={}&data=tmpf&data=p01m&year1=2019&month1=1&day1=1&year2=2019&month2=1&day2=2&tz=Etc%2FUTC&format=onlycomma&latlon=no&missing=M&trace=T&direct=no&report_type=1&report_type=2"
base_url2 = "https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station={}&data=tmpf&data=p01m&year1=2014&month1=1&day1=1&year2=2019&month2=1&day2=2&tz=Etc%2FUTC&format=onlycomma&latlon=no&missing=M&trace=T&direct=no&report_type=1&report_type=2"
build_dataset("gage_data",  base_url, base_url2, eco_gage_set, "record.json", 5998, 6052)

{'_base_connection': <google.cloud.storage._http.Connection object at 0x7f3ce3746b38>, 'project': 'gmap-997', '_credentials': <google.oauth2.credentials.Credentials object at 0x7f3cc0755cf8>, '_http_internal': None, '_batch_stack': <google.cloud._helpers._LocalStack object at 0x7f3cc0821b28>}


KeyboardInterrupt: ignored

In [ ]:
!ls

adc.json  gage_data  record.json  sample_data


In [ ]:
!ls distance3/gage_data

ls: cannot access 'distance3/gage_data': No such file or directory


In [ ]:
!gsutil -m cp  -r gs://predict_cfs/meta_data/distance3/ . 
!gsutil -m cp -r gs://predict_cfs/asos . 

Copying gs://predict_cfs/meta_data/distance3/1010000stations.json...
Copying gs://predict_cfs/meta_data/distance3/1010070stations.json...
Copying gs://predict_cfs/meta_data/distance3/1013500stations.json...
Copying gs://predict_cfs/meta_data/distance3/1010500stations.json...
Copying gs://predict_cfs/meta_data/distance3/1014000stations.json...
Copying gs://predict_cfs/meta_data/distance3/1011000stations.json...
Copying gs://predict_cfs/meta_data/distance3/1015800stations.json...
Copying gs://predict_cfs/meta_data/distance3/1012960stations.json...
Copying gs://predict_cfs/meta_data/distance3/1017000stations.json...
Copying gs://predict_cfs/meta_data/distance3/1017060stations.json...
Copying gs://predict_cfs/meta_data/distance3/1017290stations.json...
Copying gs://predict_cfs/meta_data/distance3/1017550stations.json...
Copying gs://predict_cfs/meta_data/distance3/1017960stations.json...
Copying gs://predict_cfs/meta_data/distance3/1018009stations.json...
Copying gs://predict_cfs/meta_data

In [ ]:
import pandas as pd 
import requests 
from datetime import datetime
from typing import Tuple, Dict
# url format 
def make_usgs_data(start_date:datetime, end_date:datetime, site_number:str):
    base_url = "https://nwis.waterdata.usgs.gov/usa/nwis/uv/?cb_00060=on&cb_00065&format=rdb&"
    full_url = base_url + "site_no=" + site_number + "&period=&begin_date="+start_date.strftime("%Y-%m-%d") + "&end_date="+end_date.strftime("%Y-%m-%d")
    print("Getting request from USGS")
    print(full_url)
    r = requests.get(full_url)
    with open(site_number + ".txt", "w") as f:
        f.write(r.text)
    print("Request finished")
    response_data = process_response_text(site_number + ".txt")
    create_csv(response_data[0], response_data[1], site_number)
    return pd.read_csv(site_number + "_flow_data.csv")

def process_response_text(file_name:str)->Tuple[str, Dict]:
    extractive_params = {}
    with open(file_name, "r") as f: 
        lines = f.readlines()
        i = 0 
        params = False 
        while "#" in lines[i]:
            # TODO figure out getting height and discharge code efficently
            the_split_line = lines[i].split()[1:]
            if params:
                print(the_split_line)
                if len(the_split_line)<2:
                    params = False 
                else:
                    extractive_params[the_split_line[0]+"_"+the_split_line[1]] = df_label(the_split_line[2])
            if len(the_split_line)>2:
                if the_split_line[0]== "TS":
                    params = True
            i+=1
        with open(file_name.split(".")[0] + "data.tsv", "w") as t:
            t.write("".join(lines[i:]))
        return file_name.split(".")[0] + "data.tsv", extractive_params

def df_label(usgs_text:str)->str:
    usgs_text = usgs_text.replace(",","")
    if usgs_text == "Discharge":
        return "cfs"
    elif usgs_text=="Gage":
        return "height"
    else: 
        return usgs_text

def create_csv(file_path:str, params_names:dict, site_number:str):
    """
    Function that creates the final version of the CSV file
    Assigns 
    """
    print(params_names)
    df = pd.read_csv(file_path, sep="\t")
    for key, value in params_names.items():
        df[value] = df[key]
    df.to_csv(site_number + "_flow_data.csv")



In [ ]:
timezone_map = {"EST": "America/New_York", "EDT":"America/New_York", "CST":"America/Chicago", "CDT":"America/Chicago", "MDT":"America/Denver", "MST":"America/Denver",
                "PST": "America/Los_Angeles", "PDT": "America/Los_Angeles"}

In [ ]:
from datetime import datetime
import os 
  
def create_usgs(meta_data_dir:str, precip_path:str, start, end):
  gage_list = sorted(os.listdir(meta_data_dir))
  exceptions = {} 
  client = get_client()
  for i in range(start, end):
    try:
      file_name = gage_list[i]
      gage_id = file_name.split("stations")[0]
      with open(os.path.join(meta_data_dir , file_name)) as f:
        print(os.path.join(meta_data_dir , file_name))
        data = json.load(f)
      if len(gage_id) == 7:
        gage_id = "0"+gage_id
        raw_df = make_usgs_data(datetime(2014, 1, 1), datetime(2019,1,1), gage_id)
      else: 
        raw_df = make_usgs_data(datetime(2014, 1, 1), datetime(2019,1,1), gage_id)
      df, max_flow, min_flow = process_intermediate_csv(raw_df)
      data["time_zone_code"] = df["tz_cd"].iloc[0]
      data["max_flow"] = max_flow
      data["min_flow"] = min_flow
      precip_df = pd.read_csv(os.path.join(precip_path, data["stations"][0]["station_id"] + ".csv"))
      fixed_df, nan_flow, nan_precip = combine_data(df, precip_df)
      data["nan_flow"] = nan_flow
      data["nan_precip"] = nan_precip
      joined_name = str(gage_id) + data["stations"][0]["station_id"] + "_flow.csv"
      joined_upload = "joined5/" + joined_name
      meta_path = os.path.join(meta_data_dir , file_name)
      data["files"] = [joined_name]
      fixed_df.to_csv(joined_name)
      with open(meta_path, 'w') as f:
        json.dump(data, f)
      upload_file("predict_cfs", "meta4/" + file_name, meta_path, client)
      upload_file("predict_cfs", joined_upload, joined_name, client)
    except Exception as e:
      exceptions[str(gage_id)] = str(e)
      with open("exceptions10.json", "w+") as a:
        json.dump(exceptions, a)
      print("exception")
      print(e)
      upload_file("predict_cfs", "meta4/" + "exceptions10.json", "exceptions10.json", client)


def combine_data(flow_df, precip_df):
  tz = pytz.timezone("UTC")
  precip_df['hour_updated'] = precip_df['hour_updated'].map(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
  precip_df['hour_updated'] = precip_df['hour_updated'].map(lambda x: tz.localize(x))
  joined_df = precip_df.merge(flow_df, left_on='hour_updated', right_on='datetime', how='outer')[5:-5]
  nan_precip = sum(pd.isnull(joined_df['p01m']))
  nan_flow = sum(pd.isnull(joined_df['cfs']))
  return joined_df, nan_flow, nan_precip

In [ ]:
import pytz
from typing import Tuple
def process_intermediate_csv(df:pd.DataFrame) -> (pd.DataFrame, int, int, int):
  # Remove garbage first row
  # TODO check if more rows are garabage
  df = df.iloc[1:]
  time_zone = df["tz_cd"].iloc[0]
  time_zone = timezone_map[time_zone]
  old_timezone = pytz.timezone(time_zone)
  new_timezone = pytz.timezone("UTC")
  # This assumes timezones are consistent throughout the USGS stream (this should be true)
  df["datetime"] = df["datetime"].map(lambda x: old_timezone.localize(datetime.strptime(x, "%Y-%m-%d %H:%M")).astimezone(new_timezone))
  df["cfs"] = pd.to_numeric(df['cfs'], errors='coerce')
  max_flow = df["cfs"].max()
  min_flow = df["cfs"].min()
  count_nan = len(df["cfs"]) - df["cfs"].count()
  return df[df.datetime.dt.minute==0], max_flow, min_flow




In [ ]:
import json
create_usgs("distance3", "asos", 2600, 3000)
#df['hour_updated'] = df['hour_updated'].map(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
#tz = pytz.timezone("UTC")
#df['hour_updated'] = df['hour_updated'].map(lambda x: tz.localize(x))

{'_base_connection': <google.cloud.storage._http.Connection object at 0x7f3cc02110b8>, 'project': 'gmap-997', '_credentials': <google.oauth2.credentials.Credentials object at 0x7f3cc0116b70>, '_http_internal': None, '_batch_stack': <google.cloud._helpers._LocalStack object at 0x7f3cc01a5048>}
distance3/7148111stations.json
Getting request from USGS
https://nwis.waterdata.usgs.gov/usa/nwis/uv/?cb_00060=on&cb_00065&format=rdb&site_no=07148111&period=&begin_date=2014-01-01&end_date=2019-01-01
Request finished
['57742', '00060', 'Discharge,', 'cubic', 'feet', 'per', 'second']
[]
{'57742_00060': 'cfs'}


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (1,4) have mixed types. Specify dtype option on import or set low_memory=False.
  
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2,5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

distance3/7148111stations.json
07148111KWLD_flow.csv
distance3/7149000stations.json
Getting request from USGS
https://nwis.waterdata.usgs.gov/usa/nwis/uv/?cb_00060=on&cb_00065&format=rdb&site_no=07149000&period=&begin_date=2014-01-01&end_date=2019-01-01
Request finished
['57749', '00060', 'Discharge,', 'cubic', 'feet', 'per', 'second']
[]
{'57749_00060': 'cfs'}


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (1,4) have mixed types. Specify dtype option on import or set low_memory=False.
  


distance3/7149000stations.json
07149000P28_flow.csv
distance3/7151500stations.json
Getting request from USGS
https://nwis.waterdata.usgs.gov/usa/nwis/uv/?cb_00060=on&cb_00065&format=rdb&site_no=07151500&period=&begin_date=2014-01-01&end_date=2019-01-01
Request finished
['57757', '00060', 'Discharge,', 'cubic', 'feet', 'per', 'second']
[]
{'57757_00060': 'cfs'}
distance3/7151500stations.json
07151500KBKN_flow.csv
distance3/7155590stations.json
Getting request from USGS
https://nwis.waterdata.usgs.gov/usa/nwis/uv/?cb_00060=on&cb_00065&format=rdb&site_no=07155590&period=&begin_date=2014-01-01&end_date=2019-01-01
Request finished
['244999', '00060', 'Discharge,', 'cubic', 'feet', 'per', 'second']
[]
{'244999_00060': 'cfs'}
distance3/7155590stations.json
07155590EHA_flow.csv
distance3/7156220stations.json
Getting request from USGS
https://nwis.waterdata.usgs.gov/usa/nwis/uv/?cb_00060=on&cb_00065&format=rdb&site_no=07156220&period=&begin_date=2014-01-01&end_date=2019-01-01
Request finished
[

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (1,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2,5,7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


distance3/7385700stations.json
07385700ARA_flow.csv
distance3/7385702stations.json
Getting request from USGS
https://nwis.waterdata.usgs.gov/usa/nwis/uv/?cb_00060=on&cb_00065&format=rdb&site_no=07385702&period=&begin_date=2014-01-01&end_date=2019-01-01
Request finished
{}
exception
'tz_cd'
exceptions10.json
distance3/7385765stations.json
Getting request from USGS
https://nwis.waterdata.usgs.gov/usa/nwis/uv/?cb_00060=on&cb_00065&format=rdb&site_no=07385765&period=&begin_date=2014-01-01&end_date=2019-01-01
Request finished
['63150', '00060', 'Discharge,', 'cubic', 'feet', 'per', 'second']
[]
{'63150_00060': 'cfs'}
distance3/7385765stations.json
07385765KPTN_flow.csv
distance3/7385790stations.json
Getting request from USGS
https://nwis.waterdata.usgs.gov/usa/nwis/uv/?cb_00060=on&cb_00065&format=rdb&site_no=07385790&period=&begin_date=2014-01-01&end_date=2019-01-01
Request finished
['63160', '00060', 'Discharge,', 'cubic', 'feet', 'per', 'second']
[]
{'63160_00060': 'cfs'}
distance3/738579

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


distance3/9419507stations.json
09419507LSV_flow.csv
distance3/9419515stations.json
Getting request from USGS
https://nwis.waterdata.usgs.gov/usa/nwis/uv/?cb_00060=on&cb_00065&format=rdb&site_no=09419515&period=&begin_date=2014-01-01&end_date=2019-01-01
Request finished
['209576', '00060', 'Discharge,', 'cubic', 'feet', 'per', 'second']
[]
{'209576_00060': 'cfs'}
exception
'209576_00060'
exceptions10.json
distance3/9419530stations.json
Getting request from USGS
https://nwis.waterdata.usgs.gov/usa/nwis/uv/?cb_00060=on&cb_00065&format=rdb&site_no=09419530&period=&begin_date=2014-01-01&end_date=2019-01-01
Request finished
['247144', '00065', 'Gage', 'height,', 'feet']
['247145', '00060', 'Discharge,', 'cubic', 'feet', 'per', 'second']
[]
{'247144_00065': 'height', '247145_00060': 'cfs'}
exception
'247144_00065'
exceptions10.json
distance3/9419547stations.json
Getting request from USGS
https://nwis.waterdata.usgs.gov/usa/nwis/uv/?cb_00060=on&cb_00065&format=rdb&site_no=09419547&period=&begi

In [ ]:
df.write_csv()

,Unnamed: 0_x,hour_updated,p01m,valid,tmpf,Unnamed: 0_y,agency_cd,site_no,datetime,tz_cd,194445_00060,194445_00060_cd,cfs
43618,43613.0,2018-12-30 04:00:00+00:00,0.0,2018-12-30 03:05,38.846154,174987.0,USGS,1388500,2018-12-30 04:00:00+00:00,EST,1820,P,1820.0
43619,43614.0,2018-12-30 05:00:00+00:00,0.0,2018-12-30 04:05,37.769231,174991.0,USGS,1388500,2018-12-30 05:00:00+00:00,EST,1800,P,1800.0
43620,43615.0,2018-12-30 06:00:00+00:00,0.0,2018-12-30 05:05,36.769231,174995.0,USGS,1388500,2018-12-30 06:00:00+00:00,EST,1780,P,1780.0
43621,43616.0,2018-12-30 07:00:00+00:00,0.0,2018-12-30 06:05,35.000000,174999.0,USGS,1388500,2018-12-30 07:00:00+00:00,EST,1760,P,1760.0
43622,43617.0,2018-12-30 08:00:00+00:00,0.0,2018-12-30 07:05,32.853846,175003.0,USGS,1388500,2018-12-30 08:00:00+00:00,EST,1730,P,1730.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28824,28821.0,2017-04-20 19:00:00+00:00,0.0,2017-04-20 18:05,64.000000,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
31446,31443.0,2017-08-08 16:00:00+00:00,0.0,2017-08-08 15:05,71.461538,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
32122,32119.0,2017-09-05 20:00:00+00:00,0.0,2017-09-05 19:05,85.846154,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
33571,33568.0,2017-11-05 05:00:00+00:00,0.0,2017-11-05 04:05,48.553846,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN


In [ ]:
def compute_meta_stats(df:pd.DataFrame):
  pass
def compute_elev(meta_data):
  pass 

In [ ]:
df.count()

Unnamed: 0      43682
hour_updated    43682
p01m            43682
valid           43682
tmpf            43682
dtype: int64

In [ ]:
with open("exceptions.json") as f:
  data = json.load(f)
len(data.keys())

165

In [ ]:
make_usgs_data(datetime(2014,1,1), datetime(2019,1,1), "06894000")

Getting request from USGS
https://nwis.waterdata.usgs.gov/usa/nwis/uv/?cb_00060=on&cb_00065&format=rdb&site_no=06894000&period=&begin_date=2014-01-01&end_date=2019-01-01
Request finished
['76992', '00060', 'Discharge,', 'cubic', 'feet', 'per', 'second']
[]
{'76992_00060': 'cfs'}


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1,4) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2828: DtypeWarning: Columns (2,5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,Unnamed: 0,agency_cd,site_no,datetime,tz_cd,76992_00060,76992_00060_cd,cfs
0,0,5s,15s,20d,6s,14n,10s,14n
1,1,USGS,06894000,2014-01-01 00:00,CST,27.8,A,27.8
2,2,USGS,06894000,2014-01-01 00:15,CST,27.8,A,27.8
3,3,USGS,06894000,2014-01-01 00:30,CST,27.8,A,27.8
4,4,USGS,06894000,2014-01-01 00:45,CST,27.8,A,27.8
...,...,...,...,...,...,...,...,...
171642,171642,USGS,6894000,2019-01-01 22:45,CST,391,A,391
171643,171643,USGS,6894000,2019-01-01 23:00,CST,391,A,391
171644,171644,USGS,6894000,2019-01-01 23:15,CST,387,A,387
171645,171645,USGS,6894000,2019-01-01 23:30,CST,387,A,387


In [ ]:
min_flow

21.1

In [ ]:
ol_timezone = pytz.timezone(timezone_map[df['tz_cd'].iloc[1]])
new_timezone = pytz.timezone("UTC")
ol_timezone.localize(datetime.strptime(df['datetime'].iloc[1], "%Y-%m-%d %H:%M")).astimezone(new_timezone)

datetime.datetime(2014, 4, 1, 6, 0, tzinfo=<UTC>)

In [ ]:
df.head()

,Unnamed: 0,agency_cd,site_no,datetime,tz_cd,82505_00060,82505_00060_cd,cfs
0,0,5s,15s,20d,6s,14n,10s,14n
1,1,USGS,06127500,2014-04-01 00:00,MDT,200,A,200
2,2,USGS,06127500,2014-04-01 00:15,MDT,200,A,200
3,3,USGS,06127500,2014-04-01 00:30,MDT,200,A,200
4,4,USGS,06127500,2014-04-01 00:45,MDT,200,A,200


In [ ]:
df = pd.read_csv("06127500_flow_data.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df

,Unnamed: 0,agency_cd,site_no,datetime,tz_cd,82505_00060,82505_00060_cd,cfs
0,0,5s,15s,20d,6s,14n,10s,14n
1,1,USGS,06127500,2014-04-01 00:00,MDT,200,A,200
2,2,USGS,06127500,2014-04-01 00:15,MDT,200,A,200
3,3,USGS,06127500,2014-04-01 00:30,MDT,200,A,200
4,4,USGS,06127500,2014-04-01 00:45,MDT,200,A,200
...,...,...,...,...,...,...,...,...
104733,104733,USGS,6127500,2018-11-01 12:30,MDT,262,A,262
104734,104734,USGS,6127500,2018-11-01 12:45,MDT,265,P,265
104735,104735,USGS,6127500,2018-11-01 13:00,MDT,268,P,268
104736,104736,USGS,6127500,2018-11-01 13:15,MDT,265,P,265


In [ ]:
data[data.datetime.dt.minute==0]

,Unnamed: 0,agency_cd,site_no,datetime,tz_cd,194445_00060,194445_00060_cd,cfs
1,1,USGS,01388500,2014-01-01 05:00:00+00:00,EST,606,A,606.0
5,5,USGS,01388500,2014-01-01 06:00:00+00:00,EST,601,A,601.0
9,9,USGS,01388500,2014-01-01 07:00:00+00:00,EST,590,A,590.0
13,13,USGS,01388500,2014-01-01 08:00:00+00:00,EST,584,A,584.0
17,17,USGS,01388500,2014-01-01 09:00:00+00:00,EST,579,A,579.0
...,...,...,...,...,...,...,...,...
175259,175259,USGS,1388500,2019-01-02 00:00:00+00:00,EST,2650,P,2650.0
175263,175263,USGS,1388500,2019-01-02 01:00:00+00:00,EST,2620,P,2620.0
175267,175267,USGS,1388500,2019-01-02 02:00:00+00:00,EST,2600,P,2600.0
175271,175271,USGS,1388500,2019-01-02 03:00:00+00:00,EST,2570,P,2570.0


In [ ]:
pre[pre['hour_updated'] == datetime()

,Unnamed: 0,hour_updated,p01m,valid,tmpf
0,0,2014-01-01 01:00:00+00:00,0.0,2014-01-01 00:53,28.040000
1,1,2014-01-01 02:00:00+00:00,0.0,2014-01-01 01:53,26.960000
2,2,2014-01-01 03:00:00+00:00,0.0,2014-01-01 02:53,26.060000
3,3,2014-01-01 04:00:00+00:00,0.0,2014-01-01 03:53,24.080000
4,4,2014-01-01 05:00:00+00:00,0.0,2014-01-01 04:53,24.080000
...,...,...,...,...,...
43677,43677,2019-01-01 20:00:00+00:00,0.0,2019-01-01 19:05,49.100000
43678,43678,2019-01-01 21:00:00+00:00,0.0,2019-01-01 20:05,47.038462
43679,43679,2019-01-01 22:00:00+00:00,0.0,2019-01-01 21:05,45.315385
43680,43680,2019-01-01 23:00:00+00:00,0.0,2019-01-01 22:05,44.361538


In [ ]:
df.to_csv("example_csv.csv")

In [ ]:
!gsutil cp example_csv.csv gs://predict_cfs/combined_data/example_csv.csv

Copying file://example_csv.csv [Content-Type=text/csv]...
/ [1 files][  5.7 MiB/  5.7 MiB]                                                
Operation completed over 1 objects/5.7 MiB.                                      


In [ ]:
count_nan = len(df) - df.count()

In [ ]:
count_nan

Unnamed: 0_x       163
hour_updated       163
p01m               163
valid              163
tmpf               163
Unnamed: 0_y        31
agency_cd           31
site_no             31
datetime            31
tz_cd               31
194445_00060        31
194445_00060_cd     31
cfs                 31
dtype: int64

In [ ]:
count_nan_precip = len(pre) - pre.count()
count_nan_precip

Unnamed: 0      0
hour_updated    0
p01m            0
valid           0
tmpf            0
dtype: int64

In [ ]:
len(pre)

43682

In [ ]:
df

,Unnamed: 0,agency_cd,site_no,datetime,tz_cd,194445_00060,194445_00060_cd,cfs
1,1,USGS,01388500,2014-01-01 05:00:00+00:00,EST,606,A,606.0
5,5,USGS,01388500,2014-01-01 06:00:00+00:00,EST,601,A,601.0
9,9,USGS,01388500,2014-01-01 07:00:00+00:00,EST,590,A,590.0
13,13,USGS,01388500,2014-01-01 08:00:00+00:00,EST,584,A,584.0
17,17,USGS,01388500,2014-01-01 09:00:00+00:00,EST,579,A,579.0
...,...,...,...,...,...,...,...,...
175259,175259,USGS,1388500,2019-01-02 00:00:00+00:00,EST,2650,P,2650.0
175263,175263,USGS,1388500,2019-01-02 01:00:00+00:00,EST,2620,P,2620.0
175267,175267,USGS,1388500,2019-01-02 02:00:00+00:00,EST,2600,P,2600.0
175271,175271,USGS,1388500,2019-01-02 03:00:00+00:00,EST,2570,P,2570.0


In [ ]:
joined_df['p01m']=joined_df.fillna(method='ffill').fillna(method='bfill')

NameError: ignored

In [ ]:
joined_df[joined_df['p01m'].isna()].sort_values(by='datetime')

,Unnamed: 0_x,hour_updated,p01m,valid,tmpf,Unnamed: 0_y,agency_cd,site_no,datetime,tz_cd,194445_00060,194445_00060_cd,cfs


In [ ]:
joined_d['p01m']=(df['p01m'].fillna(method='ffill') + df['p01m'].fillna(method='bfill'))/2

In [ ]:
joined_df.set_index('datetime').sort_index().tail(40)

,Unnamed: 0_x,hour_updated,p01m,valid,tmpf,Unnamed: 0_y,agency_cd,site_no,tz_cd,194445_00060,194445_00060_cd,cfs
datetime,,,,,,,,,,,,
2019-01-01 16:00:00+00:00,43673.0,2019-01-01 16:00:00+00:00,0.0,2019-01-01 15:05,56.769231,175227.0,USGS,1388500,EST,2690,P,2690.0
2019-01-01 17:00:00+00:00,43674.0,2019-01-01 17:00:00+00:00,0.0,2019-01-01 16:05,55.084615,175231.0,USGS,1388500,EST,2670,P,2670.0
2019-01-01 18:00:00+00:00,43675.0,2019-01-01 18:00:00+00:00,0.0,2019-01-01 17:05,53.207692,175235.0,USGS,1388500,EST,2650,P,2650.0
2019-01-01 19:00:00+00:00,43676.0,2019-01-01 19:00:00+00:00,0.0,2019-01-01 18:05,51.107692,175239.0,USGS,1388500,EST,2650,P,2650.0
2019-01-01 20:00:00+00:00,43677.0,2019-01-01 20:00:00+00:00,0.0,2019-01-01 19:05,49.100000,175243.0,USGS,1388500,EST,2650,P,2650.0
2019-01-01 21:00:00+00:00,43678.0,2019-01-01 21:00:00+00:00,0.0,2019-01-01 20:05,47.038462,175247.0,USGS,1388500,EST,2650,P,2650.0
2019-01-01 22:00:00+00:00,43679.0,2019-01-01 22:00:00+00:00,0.0,2019-01-01 21:05,45.315385,175251.0,USGS,1388500,EST,2650,P,2650.0
2019-01-01 23:00:00+00:00,43680.0,2019-01-01 23:00:00+00:00,0.0,2019-01-01 22:05,44.361538,175255.0,USGS,1388500,EST,2650,P,2650.0
2019-01-02 00:00:00+00:00,43681.0,2019-01-02 00:00:00+00:00,0.0,2019-01-01 23:05,43.500000,175259.0,USGS,1388500,EST,2650,P,2650.0


In [ ]:
with open("distance3/384456119222101stations.json") as f: 
  data = json.load(f)

In [ ]:
data

{'gage_id': 384456119222101,
 'stations': [{'cat': 'ASOS',
   'dist': 42.718108597553965,
   'missing_precip': 0,
   'missing_temp': 1184,
   'station_id': 'MEV'},
  {'cat': 'ASOS',
   'dist': 44.719722632504485,
   'missing_precip': 0,
   'missing_temp': 380,
   'station_id': 'BAN'},
  {'cat': 'ASOS',
   'dist': 45.53881794914841,
   'missing_precip': 0,
   'missing_temp': 380,
   'station_id': 'KBAN'},
  {'cat': 'ASOS',
   'dist': 56.31511546856264,
   'missing_precip': 262506,
   'missing_temp': 274514,
   'station_id': 'TVL'},
  {'cat': 'ASOS',
   'dist': 56.403195751932756,
   'missing_precip': 262506,
   'missing_temp': 274514,
   'station_id': 'KTVL'},
  {'cat': 'ASOS',
   'dist': 57.45852353307815,
   'missing_precip': 0,
   'missing_temp': 46,
   'station_id': 'KCXP'},
  {'cat': 'ASOS',
   'dist': 58.310412686357125,
   'missing_precip': 0,
   'missing_temp': 46,
   'station_id': 'CXP'},
  {'cat': 'ASOS',
   'dist': 68.23565331011748,
   'missing_precip': 0,
   'missing_temp':

In [ ]:
!ls

01010000data.tsv	0384456119222101data.tsv
01010000_flow_data.csv	0384456119222101.txt
01010000.txt		10100001010000stations.json_flow.csv
01010070data.tsv	10100701010070stations.json_flow.csv
01010070_flow_data.csv	13885001388500stations.json_flow.csv
01010070.txt		adc.json
01388500data.tsv	asos
01388500_flow_data.csv	distance3
01388500.txt		sample_data


In [ ]:
inspect_df = pd.read_csv("10100001010000stations.json_flow.csv").sort_values('datetime')

In [ ]:
sum(pd.isnull(inspect_df['p01m']))

729

In [ ]:
len(inspect_df)

39105

In [ ]:
# Total of 43,800
inspect_df2 = pd.read_csv("10100701010070stations.json_flow.csv").sort_values('datetime')

In [ ]:
len(inspect_df2)

39054

In [ ]:
sum(pd.isnull(inspect_df2['p01m']))

678

In [ ]:
sum(pd.isnull(inspect_df2['cfs']))

13627

In [ ]:
!ls asos 

04W.csv   CWNL.csv  HZR.csv   KCXY.csv	KLCI.csv  KSPW.csv  OLG.csv
05U.csv   CWOE.csv  HZX.csv   KCYS.csv	KLDJ.csv  KSRB.csv  OLU.csv
0R0.csv   CWPE.csv  I16.csv   KD25.csv	KLDM.csv  KSTC.csv  OLV.csv
0R4.csv   CWQH.csv  I35.csv   KD39.csv	KLEB.csv  KSTF.csv  OMA.csv
12N.csv   CWQO.csv  I63.csv   KD50.csv	KLEW.csv  KSTJ.csv  ONA.csv
13K.csv   CWRT.csv  I67.csv   KD55.csv	KLEX.csv  KSTK.csv  ONL.csv
14Y.csv   CWRY.csv  I69.csv   KD60.csv	KLFT.csv  KSTL.csv  ONZ.csv
19S.csv   CWSS.csv  IAB.csv   KD95.csv	KLGA.csv  KSTP.csv  OOSA.csv
1A6.csv   CWST.csv  IAD.csv   KDAA.csv	KLHX.csv  KSUS.csv  OPL.csv
1GM.csv   CWSW.csv  IBM.csv   KDAW.csv	KLJF.csv  KSUW.csv  ORB.csv
1K1.csv   CWTN.csv  ICR.csv   KDCA.csv	KLLJ.csv  KSUX.csv  ORE.csv
1KM.csv   CWTT.csv  ICT.csv   KDDC.csv	KLLQ.csv  KSWO.csv  ORH.csv
1LM.csv   CWVN.csv  IDP.csv   KDDH.csv	KLNK.csv  KSYF.csv  OSC.csv
1M4.csv   CWVP.csv  IEN.csv   KDEH.csv	KLNL.csv  KSYN.csv  OTG.csv
1M5.csv   CWVU.csv  IER.csv   KDET.csv	KLNP.csv  KSZT.csv  OT

In [ ]:
def add_data_to_asos(asos_file_dir):
  for 

In [ ]:
!ls

adc.json  gage_data  record.json  sample_data


In [ ]:
!ls -1 gage_data | wc -l

6059
